# Tema: Pipelines de Extração, Transformação e Carga para o Relacionamento de Dados Heterogêneos

- Uma Rede Complexa de Informações Com Notação Semântica
- Estudo de Caso: Políticos do Brasil - Deputados Federais.

# Extração

https://dadosabertos.camara.leg.br/swagger/api.html


## Deputados

- Listagem e busca de deputados, segundo critérios

In [ ]:
import requests
import pandas as pd
import os

def save_to_csv(data, filepath):
    """
    Salva a lista de deputados em um arquivo CSV.
    """

    df = pd.DataFrame(data)
    df.to_csv(filepath, index=False, encoding="utf-8")
    print(f"[INFO] Arquivo CSV '{filepath}' salvo com sucesso!")

def fetch_deputados(ordem="ASC", ordenar_por="nome", pagina=1, idLegislatura=57):
    """
    Faz a requisição à API da Câmara e retorna a lista de deputados de uma página específica.
    """
    url = f"https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura={idLegislatura}&ordem={ordem}&ordenarPor={ordenar_por}&pagina={pagina}"
    headers = {
        "accept": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data.get("dados", [])
    except requests.RequestException as e:
        print(f"[ERRO] Falha na requisição: {e}")
        return []

def fetch_all_deputados(idLegislatura=57):
    """
    Faz a requisição de todas as páginas de deputados.
    """
    deputados_total = []
    pagina = 1

    while True:
        deputados = fetch_deputados(pagina=pagina, idLegislatura=idLegislatura)
        if not deputados:
            break

        deputados_total.extend(deputados)
        print(f"[INFO] Página {pagina} processada com {len(deputados)} deputados.")
        pagina += 1

    return deputados_total

def extraction_deputado(directory):
    '''Extração - Deputados Federais'''

    print("########################")
    print("## Extração - Deputados ##")
    print("########################")

    idLegislatura = "57" # Legislatura atual

    deputados = fetch_all_deputados(idLegislatura)
    filepath = os.path.join(directory, "deputados_legisl_"+idLegislatura+".csv")
    save_to_csv(deputados, filepath)

def transformation_deputado():
  '''Transformação - Deputados Federais'''
  print("########################")
  print("## Transformação - Deputados ##")
  print("########################")

  return True

def load_deputado():
  '''Carga - Deputados Federais'''
  print("########################")
  print("## Carga - Deputados ##")
  print("########################")
  return True

if __name__ == "__main__":
  # Defina o diretório onde os dados serão salvos. Para falta no Google Drive. 
  #directory = '/content/drive/MyDrive/1-Acadêmico/___IA - IFG - 2025/Disciplina/202501 - Linguagem de Programação Aplicada/_IFG - Ling. Programação - Projeto 1/dataset'

  #Para falta localmente
  directory = './dataset'

  extraction_deputado(directory)
  #transformation_deputado()
  #load_deputado()

  # Extração - Partidos



########################
## Extração - Deputados ##
########################
[INFO] Página 1 processada com 674 deputados.
[INFO] Arquivo CSV './dataset\deputados_legisl_57.csv' salvo com sucesso!


# Transformação


In [1]:
!pip install rdflib


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/565.1 kB ? eta -:--:--
   ------------------------------------- -- 524.3/565.1 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 565.1/565.1 kB 1.8 MB/s eta 0:00:00


In [2]:
!pip install rdflib-neo4j


Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   ---------------------------------------- 0/2 [neo4j]
   -------------------- -

In [ ]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
import os
# from google.colab import userdata
from rdflib_neo4j import Neo4jStore, Neo4jStoreConfig, HANDLE_VOCAB_URI_STRATEGY


# Namespaces
SCHEMA = Namespace("http://schema.org/")
POL = Namespace("http://purl.org/ontology/politico/")
BR = Namespace("https://dadosabertos.camara.leg.br/recurso/")

def load_deputados_csv(filepath): 
    """
    Carrega o CSV dos deputados em um DataFrame pandas.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"O arquivo {filepath} não foi encontrado!")
    return pd.read_csv(filepath)

def create_rdf_graph():
    """
    Inicializa e retorna um grafo RDF com prefixos.
    """
    g = Graph()
    g.bind("schema", SCHEMA)
    g.bind("pol", POL)
    g.bind("foaf", FOAF)
    g.bind("br", BR)
    return g

def add_deputado_triples(g, row):
    """
    Adiciona as triplas RDF de um deputado ao grafo,
    representando partido e UF como nós (recursos).
    """
    deputado_uri = URIRef(f"https://dadosabertos.camara.leg.br/recurso/deputado/{row['id']}")
    partido_uri = URIRef(row['uriPartido'])
    uf_uri = URIRef(f"https://dadosabertos.camara.leg.br/recurso/uf/{row['siglaUf']}")

    # Deputado
    g.add((deputado_uri, RDF.type, SCHEMA.Person))
    g.add((deputado_uri, SCHEMA.name, Literal(row['nome'])))
    g.add((deputado_uri, SCHEMA.memberOf, partido_uri))
    g.add((deputado_uri, SCHEMA.addressRegion, uf_uri))
    g.add((deputado_uri, SCHEMA.identifier, Literal(row['id'], datatype=XSD.integer)))
    g.add((deputado_uri, FOAF.page, URIRef(row['uri'])))
    g.add((deputado_uri, SCHEMA.image, URIRef(row['urlFoto'])))
    g.add((deputado_uri, POL.legislatura, Literal(row['idLegislatura'], datatype=XSD.integer)))

    if pd.notna(row.get('email')):
        g.add((deputado_uri, SCHEMA.email, Literal(row['email'])))

    # Nó Partido
    g.add((partido_uri, RDF.type, SCHEMA.Organization))
    g.add((partido_uri, SCHEMA.name, Literal(row['siglaPartido'])))

    # Nó UF
    g.add((uf_uri, RDF.type, SCHEMA.Place))
    g.add((uf_uri, SCHEMA.name, Literal(row['siglaUf'])))

def build_rdf_graph_from_dataframe(df):
    """
    Cria e retorna o grafo RDF completo a partir do DataFrame.
    """
    g = create_rdf_graph()
    for _, row in df.iterrows():
        add_deputado_triples(g, row)
    return g

def save_graph_as_nt(g, output_path):
    """
    Salva o grafo RDF em formato N-Triples.
    """
    g.serialize(destination=output_path, format="nt")
    print(f"[INFO] Arquivo N-Triples '{output_path}' salvo com sucesso!")

def connect_neo4j(uri="bolt://192.168.0.48:7687", user="neo4j", password="Adsumus@9", database="neo4j"):
    """
    Conecta ao banco Neo4j usando rdflib-neo4j e retorna a store.
    """
    config = Neo4jStoreConfig(
        auth_data={"uri": uri, "user": user, "pwd": password, "database": database},
        handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.MAP,
        batching=True
    )
    return Neo4jStore(config=config)

def save_graph_to_neo4j(original_graph):
    """
    Salva o grafo RDF diretamente no Neo4j usando rdflib-neo4j.
    """
    config = Neo4jStoreConfig(
        auth_data={
            "uri": "bolt://192.168.0.48:7687",
            "user": "neo4j",
            "pwd": "Adsumus@9",
            "database": "neo4j"
        },
        # Estratégia para lidar com URIs de vocabulário
        # Se você quiser ignorar URIs de vocabulário, use IGNORE
        # Se você quiser manter URIs de vocabulário como estão, use RAW
        # handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.IGNORE,
        handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.MAP,

        batching=True
    )

    store = Neo4jStore(config=config)
    g = Graph(store=store)
    g += original_graph
    g.commit()
    g.close()
    print("[INFO] Grafo RDF salvo com sucesso no Neo4j!")


def main():
    """
    Fluxo completo: carrega CSV, cria grafo RDF e salva N-Triples.
    """

    idLegislatura = "57" # Legislatura atual'

    #directory = '/content/drive/MyDrive/1-Acadêmico/___IA - IFG - 2025/Disciplina/202501 - Linguagem de Programação Aplicada/_IFG - Ling. Programação - Projeto 1/dataset'
    filepath = os.path.join(directory, "deputados_legisl_"+idLegislatura+".csv")

    csv_path = os.path.join(directory, "deputados_legisl_"+idLegislatura+".csv")
    output_path = os.path.join(directory, "deputados_legisl_"+idLegislatura+".nt")

    df = load_deputados_csv(csv_path)
    g = build_rdf_graph_from_dataframe(df)

    # Salva como .nt
    save_graph_as_nt(g, output_path)

    # Salva no Neo4j
    save_graph_to_neo4j(g)

if __name__ == "__main__":
    main()


C:\Users\ronen\AppData\Roaming\Python\Python312\site-packages\rdflib\plugins\serializers\nt.py:41: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


[INFO] Arquivo N-Triples './dataset\deputados_legisl_57.nt' salvo com sucesso!
Uniqueness constraint on :Resource(uri) found. 
                
                
The store is now: Open
The store is now: Closed
IMPORTED 5530 TRIPLES
[INFO] Grafo RDF salvo com sucesso no Neo4j!


# Carga


In [ ]:
from rdflib import Graph
import networkx as nx
import matplotlib.pyplot as plt

def load_rdf_graph(nt_file):
    """
    Carrega o arquivo N-Triples em um grafo RDFLib.
    """
    g = Graph()
    g.parse(nt_file, format="nt")
    print(f"[INFO] Grafo RDF carregado com {len(g)} triplas.")
    return g

def convert_to_networkx(rdf_graph):
    """
    Converte grafo RDFLib em grafo NetworkX para visualização.
    """
    nx_graph = nx.DiGraph()
    for s, p, o in rdf_graph:
        s_label = str(s)
        p_label = str(p.split("/")[-1])
        o_label = str(o)
        nx_graph.add_edge(s_label, o_label, label=p_label)
    return nx_graph

def plot_graph(nx_graph, title="Visualização do Grafo RDF"):
    """
    Plota o grafo com Matplotlib.
    """
    pos = nx.spring_layout(nx_graph, seed=42)
    plt.figure(figsize=(15, 10))
    nx.draw(nx_graph, pos, with_labels=True, node_size=2000, node_color="lightblue", font_size=8, font_weight="bold", edge_color="gray")
    edge_labels = nx.get_edge_attributes(nx_graph, 'label')
    nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels, font_size=7)
    plt.title(title)
    plt.axis("off")
    plt.show()

def filter_graph_for_deputado(rdf_graph, deputado_id):
    """
    Filtra o grafo RDF para incluir apenas as triplas relacionadas a um deputado específico.
    """
    deputado_uri = URIRef(f"https://dadosabertos.camara.leg.br/recurso/deputado/{deputado_id}")
    filtered_graph = Graph()
    for s, p, o in rdf_graph:
        if s == deputado_uri:
            filtered_graph.add((s, p, o))
            #print(f"[INFO] Tripla adicionada: {s} {p} {o}")
            #print(f"[INFO] Grafo filtrado com {len(filtered_graph)} triplas.")
    return filtered_graph

def main():
    #directory = '/content/drive/MyDrive/1-Acadêmico/___IA - IFG - 2025/Disciplina/202501 - Linguagem de Programação Aplicada/_IFG - Ling. Programação - Projeto 1/dataset'
    nt_file = os.path.join(directory, "deputados_legisl_57.nt")
    deputado_id = 204445  # ID de Abílio Santana
    rdf_graph = load_rdf_graph(nt_file)
    filtered_graph = filter_graph_for_deputado(rdf_graph, deputado_id)
    nx_graph = convert_to_networkx(filtered_graph)
    plot_graph(nx_graph, title="Deputado: Fernando Mineiro")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'rdflib'